In [112]:
import pandas as pd
from utils_24 import p1_place


In [113]:
dependent_variables = ['point_victor']
X_1 = ['server']
X_2 = ['p1_break_pt','p2_break_pt','p1_place_ge','p1_place_le','p1_sets','p2_sets','p1_games','p2_games','p2_trend','p1_trend']
X_3 = ['set_no','game_no','point_no_to_start','elapsed_time','past_time']
X_4 = ['p1_net_pt','p2_net_pt','p1_ace','p2_ace','p1_winner','p2_winner','p1_break_pt_won','p2_break_pt_won','p1_break_pt_missed','p2_break_pt_missed','p1_net_pt_won','p2_net_pt_won','p1_distance_run','p2_distance_run','p1_double_fault','p2_double_fault','p1_unf_err','p2_unf_err','serve_width','serve_depth','return_depth','speed_mph','rally_count','last_time']

add_col = ['p1_trend','p2_trend','past_time','point_no_to_start']
dec_col = ['match_id','player1','player2','p1_score','p2_score','p1_place_de','temp_1','winner_shot_type','game_victor','set_victor']

importance_cols = ['p2_points_won', 'p2_trend*server', 'p1_points_won', 'game_no*server', 'p1_place_ge*server', 'p2_games', 'elapsed_time', 'p1_games*server', 'p2_trend', 'return_depth_ND', 'point_no_to_start*server', 'server', 'rally_count', 'point_no_to_start', 'serve_width_BW', 'p1_trend', 'p2_games*server', 'p2_distance_run', 'speed_mph', 'p1_distance_run', 'point_no', 'game_no', 'p2_winner', 'serve_depth_NCTL', 'p1_games', 'past_time', 'past_time*server', 'p1_unf_err', 'last_time', 'serve_no', 'set_no*server', 'p1_trend*server','point_victor']
zero_x = X_2+X_3+X_4
zero_col = [i for i in zero_x if i in importance_cols]

In [114]:
df = pd.read_csv('Wimbledon_featured_matches.csv').ffill()
df['server'].replace(2,-1,inplace=True)
df['point_victor'].replace(2,0,inplace=True)
for col_name in add_col:
    df[col_name] = None

C:\Users\16872\AppData\Local\Temp\ipykernel_142708\3741649940.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['server'].replace(2,-1,inplace=True)
C:\Users\16872\AppData\Local\Temp\ipykernel_142708\3741649940.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [115]:
df = p1_place(df)
df['elapsed_time'] = pd.to_timedelta(df['elapsed_time']).apply(lambda x: x.total_seconds()/60)
df['last_time'] = df.elapsed_time.diff().fillna(0).apply(lambda x: max(0, x))
grouped = df.groupby('match_id')
grouped_dfs = {}
for name, group_df in grouped:
    group_df[X_4] = group_df[X_4].shift().bfill()
    group_df = pd.get_dummies(group_df, columns=['serve_width','serve_depth','return_depth'], drop_first=True)

    group_df['p1_trend'] = group_df['point_victor'].replace(2,-1).cumsum().shift().fillna(0)
    group_df['p2_trend'] = group_df['point_victor'].replace({2:1,1:-1}).cumsum().shift().fillna(0)
    group_df['temp_1'] = ((group_df['p1_score'] == '0') & (group_df['p2_score'] == '0')).astype(int)
    last_index_1_time = 0
    last_index_1_point = 0
    for index, row in group_df.iterrows():
        if row['temp_1'] == 1:
            last_index_1_time = row['elapsed_time']
            group_df.at[index, 'past_time'] = 0
            last_index_1_point = row['point_no']
            group_df.at[index, 'point_no_to_start'] = 0
        else:
            group_df.loc[index, 'past_time'] = row['elapsed_time'] - last_index_1_time
            group_df.loc[index, 'point_no_to_start'] = row['point_no'] - last_index_1_point
    
    group_df[['serve_width_BC', 'serve_width_BW', 'serve_width_C', 'serve_width_W', 'serve_depth_NCTL',
              'return_depth_ND','point_no_to_start']] = group_df[['serve_width_BC', 'serve_width_BW', 'serve_width_C', 'serve_width_W',
                                                                  'serve_depth_NCTL','return_depth_ND','point_no_to_start']].astype(int)
    group_df['past_time'] = group_df['past_time'].astype(float)
    group_df.drop(columns=dec_col,inplace=True)
    for col in X_2:
        new_col_name = col + '*server'
        group_df[new_col_name] = df['server'] * group_df[col]
    for col in X_3:
        new_col_name = col + '*server'
        group_df[new_col_name] = df['server'] * group_df[col]
    # group_df = group_df[importance_cols]
    # group_df[zero_col] = 0
    grouped_dfs[name] = group_df



In [117]:
predict_list = ['2023-wimbledon-1601','2023-wimbledon-1306','2023-wimbledon-1402']
train_list = df.match_id.unique().tolist()
train_list.remove('2023-wimbledon-1601')
train_list.remove('2023-wimbledon-1306')
train_list.remove('2023-wimbledon-1402')
sel_train = []
for i in train_list:
    if i in grouped_dfs:
        sel_train.append(grouped_dfs[i])

In [118]:

temp = pd.concat(sel_train)
# temp = pd.concat(list(grouped_dfs.values())[:10])

X_train = temp.drop('point_victor',axis=1)
y_train = temp['point_victor']

X_test = grouped_dfs['2023-wimbledon-1306'].drop('point_victor',axis=1)
y_test = grouped_dfs['2023-wimbledon-1306']['point_victor']

In [119]:
# X_train.to_csv('data/factor_train.csv',index=False)
# y_train.to_csv('data/y_train_dec.csv',index=False)
X_test.to_csv('data/X_test_1306.csv',index=False)
y_test.to_csv('data/y_test_1306.csv',index=False)

In [120]:
X_train

,elapsed_time,set_no,game_no,point_no,p1_sets,p2_sets,p1_games,p2_games,server,serve_no,...,p1_sets*server,p2_sets*server,p1_games*server,p2_games*server,past_time*server,p2_trend*server,p1_trend*server,set_no*server,game_no*server,point_no_to_start*server
0,0.000000,1,1,1,0,0,0,0,1,2,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,0.0
1,0.633333,1,1,2,0,0,0,0,1,1,...,0.0,0.0,0.0,0.0,0.633333,0.0,0.0,1.0,1.0,1.0
2,1.016667,1,1,3,0,0,0,0,1,1,...,0.0,0.0,0.0,0.0,1.016667,-1.0,1.0,1.0,1.0,2.0
3,1.516667,1,1,4,0,0,0,0,1,1,...,0.0,0.0,0.0,0.0,1.516667,-1.0,1.0,1.0,1.0,3.0
4,2.350000,1,1,5,0,0,0,0,1,1,...,0.0,0.0,0.0,0.0,2.350000,-2.0,2.0,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7279,279.966667,5,10,330,2,2,5,4,1,1,...,2.0,2.0,5.0,4.0,0.450000,-164.0,164.0,5.0,10.0,1.0
7280,280.716667,5,10,331,2,2,5,4,1,1,...,2.0,2.0,5.0,4.0,1.200000,-165.0,165.0,5.0,10.0,2.0
7281,281.333333,5,10,332,2,2,5,4,1,1,...,2.0,2.0,5.0,4.0,1.816667,-166.0,166.0,5.0,10.0,3.0
7282,281.866667,5,10,333,2,2,5,4,1,1,...,2.0,2.0,5.0,4.0,2.350000,-166.0,166.0,5.0,10.0,4.0
